# OpEn Rust Examples: with General Gradient Function

In this example, we are going to use a function that can obtain the gradient of any given function. This sort of function was used in relaxed_ik rust version. Now, we are trying to use this approach for [the previous example that we implemented before](https://github.com/inmo-jang/optimisation_tutorial/blob/master/tools_examples/OpEn/examples_rust/OpEn_Rust_examples_obs_avoidance_simplified.ipynb).




## Problem Formulation (Remind from the previous example)

Minimise $$f(\mathbf{u}) = u_2$$ 
<div style="text-align: right"> (P1) </div>

subject to 

$$ \psi_{O}(\mathbf{x}) =  [1 - (\mathbf{u} - \mathbf{c})^{\top}(\mathbf{u} - \mathbf{c})]_{+} = 0$$
<div style="text-align: right"> (P1C) </div> 

$$ u_2 = p_1 \cdot (u_1 - p_2)^2 + p_3$$
<div style="text-align: right"> (P2C) </div> 


## OpEn Implementation

First, we need to import "optimization_engine". Also, we use "nalgebra" for linear algebra calculation. 
- In your local PC, it should be also declared in "Cargo.toml".
- Instead, in this jupyter notebook, we need to have "extern crate" as follows. 


In [2]:
extern crate optimization_engine;
use optimization_engine::{
    alm::*,
    constraints::*, panoc::*, *
};
extern crate nalgebra;
use nalgebra::base::{*};
// use nalgebra::base::{Matrix4, Matrix4x2, Matrix4x1};
// use std::cmp;

#### Problem Master Class

You should note that `AlmFactory` should have `f` and `df` being with `
fn f(u: &[f64], cost: &mut f64) -> Result<(), SolverError>` and `fn df(u: &[f64], grad: &mut [f64]) -> Result<(), SolverError>`, respectively. It means that it could be nicer if we have a master class that can simply turn out `f` or `df` values. Such an architecture is used in [`relaxed_ik` rust version](https://github.com/uwgraphics/relaxed_ik/blob/dev/src/RelaxedIK_Rust/src/bin/lib/groove/objective_master.rs), which is a good example to be worth having a look. In this example as well, we are going to implement a problem master class as follows. 


In [63]:
pub struct ProblemMaster{
    p_obs: Matrix2x1<f64>, // Obstacle Position
    p: Vec<f64> // parameters (slice)
}

impl ProblemMaster{
    pub fn init(_p: Vec<f64>, _p_obs: Matrix2x1<f64>) -> Self {
        let p = _p;
        let p_obs = _p_obs;
        Self{p, p_obs}            
    }
    
    // Cost function
    pub fn f_call(&self, u: &[f64]) -> f64{
        let cost = u[1];
        cost
    }
    
    pub fn f(&self, u: &[f64], cost: &mut f64){
        *cost = self.f_call(u);        
    }
    
    // Gradient of the cost function
    pub fn df(&self, u: &[f64], grad: &mut [f64]){
        let mut f_0 = self.f_call(u);
        
        for i in 0..u.len() {
            let mut u_h = u.to_vec();
            u_h[i] += 0.000001;
            let f_h = self.f_call(u_h.as_slice());
            grad[i] = (-f_0 + f_h) / 0.000001;
        }
       
    } 
    
    // F1 Constraint
    pub fn f1_call(&self, u: &[f64])-> Vec<f64> {
        let mut f1u = vec![0.0; u.len()];    
        f1u[0] = (1.0 - (u[0]-self.p_obs[(0,0)]).powi(2) - (u[1]-self.p_obs[(1,0)]).powi(2) ).max(0.0);
        f1u[1] = self.p[0]*(u[0] - self.p[1]).powi(2) + self.p[2] - u[1];
        return f1u;
    }
    
    pub fn f1(&self, u: &[f64], f1u: &mut [f64]){
        let mut f1u_vec = self.f1_call(u); 
        for i in 0..f1u_vec.len(){
            f1u[i] = f1u_vec[i];
        }
    }    
    
    // Jacobian of F1
    pub fn jf1_call(&self, u: &[f64])-> Matrix2<f64> {
        let mut jf1 = Matrix2::new(0.0, 0.0,
                              0.0, 0.0);
        
        let mut f1_0 = self.f1_call(u);

        for i in 0..f1_0.len(){
            for j in 0..u.len() {
                let mut u_h = u.to_vec();
                u_h[j] += 0.000001;
                let f_h = self.f1_call(u_h.as_slice());
                jf1[(i,j)] = (-f1_0[i] + f_h[i]) / 0.000001;
            }                        
        }

        return jf1;        
    } 
    
    // Jacobian Product (JF_1^{\top}*d)
    pub fn f1_jacobian_product(&self, u: &[f64], d: &[f64], res: &mut [f64]){
        let test = self.f1_call(u);
        
        let mut jf1_matrix = self.jf1_call(u);
        if test[0] < 0.0{ // Outside the obstacle
            jf1_matrix[(0,0)] = 0.0;
            jf1_matrix[(0,1)] = 0.0;  
        }          
        
        let mut d_matrix = Matrix2x1::new(0.0, 0.0);
        for i in 0..d.len(){
            d_matrix[(i,0)] = d[i];
        }
        
        let mut res_matrix =  jf1_matrix.transpose()*d_matrix;
        
        res[0] = res_matrix[(0,0)];
        res[1] = res_matrix[(1,0)];  
    }
}

#### Main function


In [71]:
fn main(_p: &[f64], _centre: &[f64]) {
    /// ===========================================
    let mut p_obs = Matrix2x1::new(0.0, 0.0);
    for i in 0.._centre.len(){
        p_obs[(i,0)] = _centre[i];
    }
    
    let mut p: Vec<f64> = Vec::new();        
    for i in 0.._p.len(){
        p.push(_p[i]);
    }
    
    let mut pm = ProblemMaster::init(p, p_obs);
    
    /// ===========================================
    
    let tolerance = 1e-5;
    let nx = 2; // problem_size: dimension of the decision variables
    let n1 = 2; // range dimensions of mappings F1
    let n2 = 0; // range dimensions of mappings F2
    let lbfgs_mem = 5; // memory of the LBFGS buffer
    
    // PANOCCache: All the information needed at every step of the algorithm
    let panoc_cache = PANOCCache::new(nx, tolerance, lbfgs_mem);
    
    // AlmCache: A cache structure that contains all the data 
    // that make up the state of the ALM/PM algorithm
    // (i.e., all those data that the algorithm updates)
    let mut alm_cache = AlmCache::new(panoc_cache, n1, n2);

    let set_c = Zero::new(); // Set C
    let bounds = Ball2::new(None, 100.0); // Set U
    let set_y = Ball2::new(None, 1e12);  // Set Y

    // ============= 
    // Re-define the functions linked to user parameters
    let f = |u: &[f64], cost: &mut f64| -> Result<(), SolverError> {
        pm.f(u, cost);
        Ok(())
    };
    
    let df = |u: &[f64], grad: &mut [f64]| -> Result<(), SolverError> {
        pm.df(u, grad);
        Ok(())
    };
    
    let f1 = |u: &[f64], f1u: &mut [f64]| -> Result<(), SolverError> {
        pm.f1(u, f1u);
        Ok(())
    };    
    
    let f1_jacobian_product = |u: &[f64], d: &[f64], res: &mut [f64]| -> Result<(), SolverError> {
        pm.f1_jacobian_product(u,d,res);
        Ok(())
    };      
    // ==============
    
    // AlmFactory: Prepare function psi and its gradient 
    // given the problem data such as f, del_f and 
    // optionally F_1, JF_1, C, F_2
    let factory = AlmFactory::new(
        f, // Cost function
        df, // Cost Gradient
        Some(f1), // MappingF1
        Some(f1_jacobian_product), // Jacobian Mapping F1 Trans
        NO_MAPPING, // MappingF2
        NO_JACOBIAN_MAPPING, // Jacobian Mapping F2 Trans
        Some(set_c), // Constraint set
        n2,
    );

    // Define an optimisation problem 
    // to be solved with AlmOptimizer
    let alm_problem = AlmProblem::new(
        bounds,
        Some(set_c),
        Some(set_y),
        |u: &[f64], xi: &[f64], cost: &mut f64| -> Result<(), SolverError> {
            factory.psi(u, xi, cost)
        },
        |u: &[f64], xi: &[f64], grad: &mut [f64]| -> Result<(), SolverError> {
            factory.d_psi(u, xi, grad)
        },
        Some(f1),
        NO_MAPPING,
        n1,
        n2,
    );

    let mut alm_optimizer = AlmOptimizer::new(&mut alm_cache, alm_problem)
        .with_delta_tolerance(1e-5)
        .with_max_outer_iterations(200)
        .with_epsilon_tolerance(1e-6)
        .with_initial_inner_tolerance(1e-2)
        .with_inner_tolerance_update_factor(0.5)
        .with_initial_penalty(100.0)
        .with_penalty_update_factor(1.05)
        .with_sufficient_decrease_coefficient(0.2)
        .with_initial_lagrange_multipliers(&vec![5.0; n1]);

    let mut u = vec![0.0; nx]; // Initial guess
    let solver_result = alm_optimizer.solve(&mut u);
    let r = solver_result.unwrap();
    println!("\n\nSolver result : {:#.7?}\n", r);
    println!("Solution u = {:#.6?}", u);
}

#### Result

##### Case (1) : $\mathbf{p} = [0.5, 0.0, 0.0]$; and $\mathbf{c} = [0.0, 0.0]$


In [72]:
main(&[0.5, 0.0, 0.0], &[0.0, 0.0]);



Solver result : AlmOptimizerStatus {
    exit_status: Converged,
    num_outer_iterations: 88,
    num_inner_iterations: 1178,
    last_problem_norm_fpr: 0.0000006,
    lagrange_multipliers: Some(
        [
            5.0298889,
            3.9940012,
        ],
    ),
    solve_time: 1.2774560ms,
    penalty: 5737.3563215,
    delta_y_norm: 0.0425532,
    f2_norm: 0.0000000,
}

Solution u = [
    0.910177,
    0.414219,
]


##### Case (2) : $\mathbf{p} = [0.5, 0.0, 0.0]$; and $\mathbf{c} = [-0.25, 0.0]$



In [73]:
main(&[0.5, 0.0, 0.0], &[-0.25, 0.0]);



Solver result : AlmOptimizerStatus {
    exit_status: Converged,
    num_outer_iterations: 73,
    num_inner_iterations: 249,
    last_problem_norm_fpr: 0.0000009,
    lagrange_multipliers: Some(
        [
            5.0161011,
            5.5350137,
        ],
    ),
    solve_time: 426.3990000µs,
    penalty: 2897.7548129,
    delta_y_norm: 0.0137157,
    f2_norm: 0.0000000,
}

Solution u = [
    0.716497,
    0.256679,
]


##### Case (3) : $\mathbf{p} = [0.5, -0.5, 0.0]$; and $\mathbf{c} = [0.0, 0.0]$

In [74]:
main(&[0.5, -0.5, 0.0], &[0.0, 0.0]);



Solver result : AlmOptimizerStatus {
    exit_status: Converged,
    num_outer_iterations: 60,
    num_inner_iterations: 876,
    last_problem_norm_fpr: 0.0000001,
    lagrange_multipliers: Some(
        [
            5.0008478,
            0.8368964,
        ],
    ),
    solve_time: 951.4560000µs,
    penalty: 1463.5630916,
    delta_y_norm: 0.0116015,
    f2_norm: 0.0000000,
}

Solution u = [
    -0.992612,
    0.121341,
]


##### Case (4) : $\mathbf{p} = [0.1, 0.0, 0.0]$; and $\mathbf{c} = [0.5, 0.0]$

In [75]:
main(&[0.1, 0.0, 0.0], &[0.5, 0.0]);



Solver result : AlmOptimizerStatus {
    exit_status: Converged,
    num_outer_iterations: 97,
    num_inner_iterations: 334,
    last_problem_norm_fpr: 0.0000001,
    lagrange_multipliers: Some(
        [
            5.0000000,
            0.9943379,
        ],
    ),
    solve_time: 663.6030000µs,
    penalty: 9345.5488840,
    delta_y_norm: 0.0297599,
    f2_norm: 0.0000000,
}

Solution u = [
    -0.499689,
    0.024972,
]


##### Case (5) : $\mathbf{p} = [-0.5, -0.5864, 0.0]$; and $\mathbf{c} = [-
3.0, 2.0]$

In [76]:
main(&[0.5, -0.5864, 0.0], &[-3.0, 2.0]);



Solver result : AlmOptimizerStatus {
    exit_status: Converged,
    num_outer_iterations: 15,
    num_inner_iterations: 18,
    last_problem_norm_fpr: 0.0000003,
    lagrange_multipliers: Some(
        [
            5.0000000,
            0.9999921,
        ],
    ),
    solve_time: 44.3680000µs,
    penalty: 179.5856326,
    delta_y_norm: 0.0000141,
    f2_norm: 0.0000000,
}

Solution u = [
    -0.586364,
    -0.000000,
]


In this case, the circle constraint does not cover the minimum point of the cost function. Thus, the optimal value should be zero, where $u_1 = p_2$.




## Summary

- Compared with the previous example, the solver can converge in all the test cases (In the previous example, we had to tune initial guess and max_iteration number). 

- Also, solution times are less than those in the preivous example. 

- We do not need to manually implement the gradients of the cost function and the constraints. 

## Future work

- Let's make the `ProblemMaster` have resizable parameters (e.g. not binding to `Matrix2x1`). 

- Let's implement a path planner based on Model Predictive Control **without consideration of the system dynamics (only concerning the path)**

